In [27]:
#importing necessary libraries
import pandas as pd
import numpy as np
from dateutil.parser import parse

# IDENTIFYING DATE COLUMNS

In [28]:
#reading the date data
date = pd.read_csv('calendar.csv')
date

,date1,date2,date3,date4,month_and_year,word_date,day_of_week,Project Name
0,11-Aug-20,5/31/2017 0:00,7/1/2018,2016-04-29T18:38:08Z,Jan-60,2-Jan-60,Sat,Cashtie (MC)
1,10/5/2019,5/31/2017 0:00,7/2/2018,2016-04-29T16:08:27Z,Jan-60,3-Jan-60,Sun,uGovernIT
2,30/02/2016,5/31/2017 0:00,7/3/2018,2016-04-29T16:19:04Z,Jan-60,4-Jan-60,Mon,All Projects
3,29.02.2019,6/30/2017 0:00,7/4/2018,2016-04-29T17:29:31Z,Jan-60,5-Jan-60,Tue,Cashtie (MC)
4,11 nov'20,6/30/2017 0:00,7/5/2018,2016-04-29T16:07:23Z,Jan-60,6-Jan-60,Wed,uGovernIT
...,...,...,...,...,...,...,...,...
66,3/8/1960,2/26/2018 0:00,9/5/2018,2016-04-06T18:00:29Z,Jan-60,8-Mar-60,Tue,Cashtie (MC)
67,3/9/1960,2/27/2018 0:00,9/6/2018,2016-04-06T18:00:51Z,Jan-60,9-Mar-60,Wed,Cashtie (MC)
68,3/10/1960,2/28/2018 0:00,9/7/2018,2016-04-06T18:00:51Z,Jan-60,10-Mar-60,Thu,Cashtie (MC)
69,3/11/1960,3/1/2018 0:00,9/8/2018,2016-04-06T18:00:51Z,Jan-60,11-Mar-60,Fri,Office 365


In [29]:
#checking data types of the columns
date.dtypes

date1             object
date2             object
date3             object
date4             object
month_and_year    object
word_date         object
day_of_week       object
Project Name      object
dtype: object

In [30]:
#identifying date columns from the data
def identify_date_cols(df):
    """
    This function temporarily identifies date columns in the data. Final Date columns are yet to be decided.
    Input : Dataframe
    Returns : List of Datetype columns
    """
    date_cols = []
    for col in df.columns:
        tries = 1
        if df[col].dtype == "object":             #finding columns with datatype "object"
            for val in df[col]: 
                if tries<5:                       #tries = 5
                    if isinstance(val,int):
                        continue
                    try:
                        parse(val)                #trying 5 times to parse the values of a column to get a datetime value
                        date_cols.append(col)     #if the value gets successfully parsed then it means its a date column and appending the column name to a separate list
                        tries = tries+1 
                        break                     #as soon as the first value of a column gets parsed, code will break and not check other values
                    except:
                        pass

    return date_cols
            

In [31]:
def removing_invalid_dates(df,date_cols):
    """
    Removing invalid dates from the data
    Input : Dataframe, list of temporary date columns
    Returns : Dataframe with identified date columns and no invalid dates,
              Final Date columns list
    """
    final_date_cols = list(date_cols)                
    for col in date_cols:
        try:
            x = pd.to_datetime(df[col],errors = "coerce",utc = True).dt.date  #converting columns to datatype "datetime" leaving "Nat" for invalid dates
            if x.isnull().sum() == df.shape[0]:                               #some values like "sat" or "saturday" or "january" also gets parsed in dateutil(giving "Nat" as output) 
                final_date_cols.remove(col)                                   #so, removing those columns from date_columns list
                continue
            df[col] = x    
            df.dropna(subset = [col],inplace = True)                          #dropping invalid dates
            df.reset_index(drop=True,inplace = True)
        except:
            print("there's a problem with ",df[col])
    return df,final_date_cols                                                 

In [32]:
def date_diff(df):
    """
    Finding difference in adjacent date columns and storing them as separate columns
    Input : Dataframe with identified date columns and no invalid dates
    Returns : Final Dataframe with columns showing difference in adjacent date columns
    """
    date_cols = identify_date_cols(df)
    final_df,final_date_cols = removing_invalid_dates(df,date_cols)
    print("date_columns: ",final_date_cols)
    for i in range(len(final_date_cols)):
        try:
            final_df[f'{final_date_cols[i]}-{final_date_cols[i+1]}'] =  np.abs(final_df[final_date_cols[i]] - final_df[final_date_cols[i+1]])
        except:
            if len(final_date_cols)!= 1:
                final_df[f'{final_date_cols[-1]}-{final_date_cols[0]}'] = np.abs(final_df[final_date_cols[-1]] - final_df[final_date_cols[0]])
            else:
                pass
    return final_df

In [33]:
final_df = date_diff(date)
final_df

date_columns:  ['date1', 'date2', 'date3', 'date4', 'month_and_year', 'word_date']


,date1,date2,date3,date4,month_and_year,word_date,day_of_week,Project Name,date1-date2,date2-date3,date3-date4,date4-month_and_year,month_and_year-word_date,word_date-date1
0,2020-08-11,2017-05-31,2018-07-01,2016-04-29,2060-01-01,2060-01-02,Sat,Cashtie (MC),1168 days,396 days,793 days,15952 days,1 days,14388 days
1,2019-10-05,2017-05-31,2018-07-02,2016-04-29,2060-01-01,2060-01-03,Sun,uGovernIT,857 days,397 days,794 days,15952 days,2 days,14700 days
2,2020-11-11,2017-06-30,2018-07-05,2016-04-29,2060-01-01,2060-01-06,Wed,uGovernIT,1230 days,370 days,797 days,15952 days,5 days,14300 days
3,1960-01-07,2017-06-30,2018-07-06,2016-04-29,2060-01-01,2060-01-07,Thu,All Projects,20994 days,371 days,798 days,15952 days,6 days,36525 days
4,1960-01-08,2017-09-30,2018-07-07,2016-04-29,2060-01-01,2060-01-08,Fri,Cashtie (MC),21085 days,280 days,799 days,15952 days,7 days,36525 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,1960-03-08,2018-02-26,2018-09-05,2016-04-06,2060-01-01,2060-03-08,Tue,Cashtie (MC),21174 days,191 days,882 days,15975 days,67 days,36525 days
65,1960-03-09,2018-02-27,2018-09-06,2016-04-06,2060-01-01,2060-03-09,Wed,Cashtie (MC),21174 days,191 days,883 days,15975 days,68 days,36525 days
66,1960-03-10,2018-02-28,2018-09-07,2016-04-06,2060-01-01,2060-03-10,Thu,Cashtie (MC),21174 days,191 days,884 days,15975 days,69 days,36525 days
67,1960-03-11,2018-03-01,2018-09-08,2016-04-06,2060-01-01,2060-03-11,Fri,Office 365,21174 days,191 days,885 days,15975 days,70 days,36525 days
